<a href="https://colab.research.google.com/github/titokenzo/OpenRedmine/blob/master/RBF/RBF_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [40]:
!git clone https://github.com/titokenzo/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


Definição da função de base radial

In [98]:
#função de base radial gaussiana
def rbfGaussiana(x, c, s):
    #return np.exp(-1 / (2 * s**2) * (x-c)**2)
    return np.exp(-1 * ( (x-c)**2 / (2 * s**2)) )

#função de Multiquadratica Inversa
def rbfMultiquadraticaInversa(x, c, s):
    return (1 / rbfMultiquadratica(x, c, s))

def rbfMultiquadratica(x, c, s):
    result = np.sqrt((s**2) + ((x-c)**2))
    return result

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMax = np.max(dist)
  stds = np.repeat(dMax / np.sqrt(2 * k), k)
  return stds

#função de cálculo da largura do campo receptivo em que se NÃO repete o mesmo valor pra todos os neurônios
def computeDifferentStds(centers,k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMin = np.min(dist)
  stds = np.repeat(dMin / 2, k)
  return stds

2ª Etapa - Treinamento de uma Rede Neural

In [99]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=4, lr=0.01, epochs=100, rbf=rbfGaussiana, computeStds=computeEqualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida (bases)
        self.lr = lr # taxa de aprendizagem (learning rate)
        self.epochs = epochs  # número de iterações
        self.rbf = rbf # função de base radial
        self.computeStds = computeStds  #função de cálculo da largura do campo receptivo

        #inicializa os pesos
        self.w = np.random.randn(self.k,attnumber)
        #inicialisa os biasis
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias 
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        print('centers: ', self.centers)
        
        #Cálculo da largura do campo receptivo
        #2º parâmetro da rede RBF
        self.stds = self.computeStds(self.centers,self.k)
        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = 0 if F < 0 else 1
                
                #função de perda 
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))

                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                #3º Parâmetro da rede 
                self.w = self.w + self.lr * error * phi 
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            F = np.sum(F) + self.b
            out = 0 if F < 0 else 1
            y_pred.append(out)

        return np.array(y_pred)




```
# Isto está formatado como código
```

# Executando a rede neural RBF

In [112]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from matplotlib import pyplot as plt

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_pos/RBF/iris.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)
    
    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

#transforma rótulos do conjunto de treinamento em numeros pra calculo do erro
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(training_labels.values)
training_labels_transformed = labelEncoder.transform(training_labels.values)

#chama RBF
#rbfnet = RBFNet(lr=1e-2, attnumber=4, k=9, rbf=rbfGaussiana, computeStds=computeEqualStds)
#rbfnet = RBFNet(lr=1e-2, attnumber=4, k=9, rbf=rbfGaussiana, computeStds=computeDifferentStds)
#rbfnet = RBFNet(lr=1e-2, attnumber=4, k=9, rbf=rbfMultiquadraticaInversa, computeStds=computeEqualStds)
rbfnet = RBFNet(lr=1e-2, attnumber=4, k=9, rbf=rbfMultiquadraticaInversa, computeStds=computeDifferentStds)
rbfnet.fit(training_inputs.values, training_labels_transformed)

#transforma rótulos do conjunto de teste em numeros pra calculo do erro
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(test_labels.values)
test_labels_transformed = labelEncoder.transform(test_labels.values)

y_pred = rbfnet.predict(test_labels_transformed)

errorabs = abs(test_labels_transformed-y_pred)

print('error: ', np.sum(errorabs)/len(test_labels_transformed))

#-----------------------------------------------------------------



centers:  [[0.87962963 0.45833333 0.90018832 0.77777778]
 [0.11111111 0.46590909 0.06625578 0.03030303]
 [0.31944444 0.8125     0.08135593 0.07083333]
 [0.65277778 0.43333333 0.62033898 0.57083333]
 [0.51388889 0.33522727 0.70570108 0.74242424]
 [0.203125   0.60677083 0.08050847 0.07291667]
 [0.64957265 0.51602564 0.7874837  0.92628205]
 [0.41666667 0.36458333 0.54519774 0.52083333]
 [0.34201389 0.19010417 0.49364407 0.45833333]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in multiply


error:  0.6666666666666666


# Descrição Mini Projeto

Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo iris.csv. Depois, modifique a função de base radial implementada (Gaussiana) para Multiquadrática Inversa e calcule a taxa de erro.


1- Calcular a quantidade de neurônios escondidos:

a) 3

b) 5

c) 7

d) 9

Qual foi a melhor configuração? Avaliaria um outro valor?
<pre>
R. Gaussiana
  3 = 0.666
  5 = 0.266
  7 = 0.266
  9 = 0.266
Multiquadrática Inversa
  3 = 0.666
  5 = 0.666
  7 = 0.666
  9 = 0.666

Para a configuração Gaussiana com 5 neurônios, já obtivemos a taxa 0.266.

A taxa não parece melhorar com o aumento dos neurônios escondidos, 
logo não faz sentido testar novas configurações.
</pre>

2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial, em que cada neurônio possui sua própria largura. 
<pre>
R. Utilizando a base Gaussiana, com 5 ou mais neurônios escondidos, 
mas com larguras próprias, a taxa aumentou para 0.666.

rbfnet = RBFNet(lr=1e-2, attnumber=4, k=9, rbf=rbfGaussiana, computeStds=computeDifferentStds)
error: 0.6666666
</pre>

3- Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana

b) Multiquadrática Inversa


Qual foi a melhor configuração?
<pre>
A Gaussiana apresentou melhores resultados, com neurônios acima de 5 e mesma largura de campo.

rbfnet = RBFNet(lr=1e-2, attnumber=4, k=5, rbf=rbfGaussiana, computeStds=computeEqualStds)
error: 0.2666666
</pre>



DATA DE ENTREGA: 25/11/2020
